In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import preprocessing as pp
from scipy import optimize as op
import scipy.io as lm

In [2]:
data = lm.loadmat('ex3data1.mat')
x = np.copy(data['X'])
y = np.copy(data['y'])
fpw = lm.loadmat('ex3weights.mat')

In [3]:
#One Hot notation of the labels 
n_values = np.max(y[:,0])+1
hot_lab = np.eye(n_values)[y[:,0]]
hot_lab = hot_lab[:,1:]

In [4]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def NN_pred(x,t1,t2):
    x_in = np.c_[np.ones((x.shape[0],1)), x]
    L2 = sigmoid(np.matmul(t1,x_in.T).T)
    L2_in = np.c_[np.ones((L2.shape[0],1)), L2]
    h = sigmoid(np.matmul(t2,L2_in.T).T)
    return h

In [5]:
#NN cost function 
def costNN(theta,x,hot_lab,lam):
    t1 = theta[0:25*401].reshape(25,401)
    t2 = theta[25*401:].reshape(10,26)
    m = x.shape[0]*1.0
    h = NN_pred(x,t1,t2)
    sum1 = -1.0 * hot_lab * np.log(h) /m
    one_m_h = np.log(1.0 - h)
    sum2 = -1.0*(1-hot_lab)* one_m_h /m
    cost  = (sum1+sum2).sum()
    regularization  = lam/(2.0*m)*(((t1**2)[:,1:]).sum()+((t2**2)[:,1:]).sum())
    return cost + regularization

In [6]:
#Backpropagation
#Sigmoid Gradient
def sigmoidGradient(z):
    return sigmoid(z)*(1.0-sigmoid(z))
def randInitializeWeights(num_in,num_out,num_hidden):
    eps = 0.12
    in1 = np.random.rand(num_hidden,num_in+1) * 2.0 *eps - eps
    in2 = np.random.rand(num_out,num_hidden+1) * 2.0 *eps - eps
    return np.concatenate((in1.ravel(),in2.ravel()))

In [7]:
# Vectorized, But needs organaizing(Sorry!)
def backpropagation(theta,x,hot_lab,lam):
    t1 = theta[0:25*401].reshape(25,401)
    t2 = theta[25*401:].reshape(10,26)
    h = NN_pred(x,t1,t2)
    m = x.shape[0]*1.0
    x_in = np.c_[np.ones((x.shape[0],1)), x]
    delta3 = h - hot_lab
    z = np.matmul(x_in,t1.T)
    z_in = np.c_[np.ones((z.shape[0],1)), z]
    delta2 = (np.matmul(delta3,t2)* sigmoidGradient(z_in))[:,1:]
    a = sigmoid(z_in)
    a[:,0] = np.ones(x.shape[0])
    grad2 = np.matmul(delta3.T,a)/m 
    grad2[:,1:] = grad2[:,1:]+ (lam/m)*t2[:,1:]
    grad1 = np.matmul(delta2.T,x_in)/m
    grad1[:,1:] = grad1[:,1:]+ (lam/m)*t1[:,1:]
    return np.concatenate((grad1.ravel(),grad2.ravel()))

In [8]:
#Gradient Chack --> Numerically checking our backpropagation function
check = False
if check:
    eps = 10.0**(-4)
    unrolled = np.concatenate((t1.ravel(),t2.ravel()))
    grad_check = np.zeros(len(unrolled))
    for i in range(0,len(unrolled)):
        T_pos = np.copy(unrolled)
        T_pos[i] = T_pos[i]+eps
        T_neg = np.copy(unrolled)
        T_neg[i] = T_neg[i]-eps
        t1_pos = T_pos[0:t1.size].reshape(t1.shape[0],t1.shape[1])
        t2_pos = T_pos[(t1.size):].reshape(t2.shape[0],t2.shape[1])
        t1_neg = T_neg[0:(t1.size)].reshape(t1.shape[0],t1.shape[1])
        t2_neg = T_neg[(t1.size):].reshape(t2.shape[0],t2.shape[1])
        grad_check[i] = (costNN(t1_pos,t2_pos,x,hot_lab,0.0) - costNN(t1_neg,t2_neg,x,hot_lab,0.0))/(2.0*eps)

In [9]:
lam = 1.0
init = randInitializeWeights(400,10,25)
res = op.minimize(costNN,init , args=(x,hot_lab,lam),method = 'Newton-CG', 
                  jac=backpropagation, options={'maxiter':50, "disp":True})

         Current function value: 0.305322
         Iterations: 50
         Function evaluations: 65
         Gradient evaluations: 3128
         Hessian evaluations: 0


In [10]:
T1 = res.x[0:25*401].reshape(25,401)
T2 = res.x[25*401:].reshape(10,26)
p = NN_pred(x,T1,T2)
prediction = p.argmax(1)+1
ac = sum(prediction == y[:,0])/5000.0

In [11]:
print "The model accuracy for test set is ",(ac*100)

The model accuracy for test set is  99.66000000000001
